# 0. Installs, imports & set-up

In [ ]:
# Steps 1 & 4
from google.cloud import bigquery
from google.oauth2 import service_account

# Steps 2 & 3
!pip install python-citybikes
# Newer version is causing issues
!pip install rpy2==3.5.1
import citybikes
import pandas as pd

# 1. Authenticate

In [ ]:
# Mount drive - needed for GBQ creds
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_path = '/content/drive/MyDrive/GitHub'

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
    f'{root_path}/tc-transit/tc-transit-81fcba2c3195.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [ ]:
# For running queries using BigQuery magic

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


# 2. Instantiate API client & make call for most recent data

In [ ]:
client = citybikes.Client()
nr = citybikes.Network(client, uid='nice-ride')

In [ ]:
nr_stations = nr.stations

# 3. Pre-process data

In [ ]:
import collections

# https://stackoverflow.com/questions/6027558/flatten-nested-dictionaries-compressing-keys
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
# Flatten nested dictionaries using recursion (function from StackOverflow)
nr_stations_flat = list(map(flatten, nr_stations['stations']))


# Stack the list of dictionaries and convert to single df
nr_df = pd.DataFrame.from_dict(nr_stations_flat)

# Don't need the prefix anymore since the dataframe is flat
nr_df.columns = nr_df.columns.str.replace('extra_', '')

# Convert 'payment-terminal' col name to 'payment_terminal'
nr_df.columns = nr_df.columns.str.replace('-', '_')

In [ ]:
nr_df = nr_df.drop(columns = ['payment', 'payment_terminal'])

# 4. Push data to GCP

In [ ]:
# Create bikes dataset
bikes = bigquery.Dataset('tc-transit.bikes')
bikes.location = 'US'
bikes = client.create_dataset(bikes)

In [ ]:
# Configure job
job_config = bigquery.LoadJobConfig(
    autodetect = True,
    source_format = bigquery.SourceFormat.CSV)


# Load dfs
job = client.load_table_from_dataframe(nr_df, 'tc-transit.bikes.stations', job_config = job_config)

In [ ]:
%%bigquery --project tc-transit

SELECT *
FROM `bikes.stations`
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,uid,id,name,latitude,longitude,empty_slots,address,ebikes,has_ebikes,last_updated,renting,returning,slots,free_bikes,timestamp
0,101,94e61a297a51c991b5885914e2449fbf,West 36th Street & Bryant Ave S,44.937750,-93.291200,7,None,0,True,1668457554,0,0,7,0,2022-12-12 14:48:14.508000+00:00
1,24,36bf2ff52269e163a2e82136016710ec,26th Street & Lyndale,44.955654,-93.288485,11,None,0,True,1668681366,0,0,11,0,2022-12-12 14:48:14.549000+00:00
2,117,d9fb85199f995d9a3310c66369ab999a,Central Ave NE & 22nd Ave NE,45.009940,-93.247114,11,None,0,True,1668423580,0,0,11,0,2022-12-12 14:48:14.760000+00:00
3,634,36feddbc0d300a24a86352af497db8c4,S 7th Street & 2nd Ave S,44.976016,-93.270397,11,None,0,True,1668536036,0,0,11,0,2022-12-12 14:48:14.694000+00:00
4,803,35c600bdfe779b4c7e6e605f8b4924d9,Cedar Lake Point Beach,44.961106,-93.324735,11,None,0,True,1668769015,0,0,11,0,2022-12-12 14:48:14.731000+00:00
5,170,e24cfd3edd4d68cf422baa3be5a5e660,Boom Island Park,44.992540,-93.270256,14,None,0,True,1669805768,0,0,15,0,2022-12-12 14:48:14.573000+00:00
6,41,dab4bfc8400924cbef8838147e72773a,Currie Park,44.969225,-93.251136,14,None,0,True,1669200344,0,0,15,0,2022-12-12 14:48:14.738000+00:00
7,214,353122afd3029edaa7f41368c29e203c,Mill City Quarter,44.980500,-93.261800,14,None,0,True,1668509979,0,0,15,0,2022-12-12 14:48:14.733000+00:00
8,953,58aa8786361dc7a2371e6319cc89916a,NE Johnson Street & E Hennepin Ave,44.991740,-93.237035,14,None,0,True,1669146813,0,0,15,0,2022-12-12 14:48:14.731000+00:00
9,37e12a58-07f8-11e7-991c-3863bb43a7d0,7db7e64897e97c69fb737dd6b49544eb,9th Street N & Hawthorne Ave,44.977314,-93.277761,15,None,0,True,1669842832,0,0,15,0,2022-12-12 14:48:14.567000+00:00
